In [30]:
import math
import matplotlib.pyplot as plt
import ROOT
import copy

from ROOT import TCanvas, TFile, TProfile, TNtuple, TH1F, TH2F, TF1, TLegend, gStyle, TF2, TCutG
from ROOT import gROOT, gBenchmark, gRandom, gSystem
ROOT.gStyle.SetOptStat(1)

import numpy as np
from uits3_krakow22.src.Event import Event
from uits3_krakow22.src.Cluster import Cluster
from uits3_krakow22.src.Track import Track
from uits3_krakow22.src.Vertex import Vertex
from uits3_krakow22.src.Utils import *

from ipywidgets import IntProgress
import matplotlib.pyplot as plt
from IPython.display import display,HTML,clear_output
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))


from skspatial.plotting import plot_3d
from skspatial.objects import Points, Cylinder, Plane

# Load Data

In [39]:
#path = "/home/cehrich/Software/uits3_krakow22/uits3_krakow22/output/yze6run456065424_221112065722.pkl" #200 MeV
#path = "/home/cehrich/Software/uits3_krakow22/uits3_krakow22/output/yze6run457063715_221113063928.pkl" #200 MeV
#path = "/home/cehrich/Software/uits3_krakow22/uits3_krakow22/output/yze6run456114514_221112120005.pkl" #80 MeV
#path = "/home/cehrich/Software/uits3_krakow22/uits3_krakow22/output/yzrun457063715_221113063928.pkl" #full file, up up
#path = "/home/cehrich/Software/uits3_krakow22/run457063715_221113063928_clusters.pkl" # full file up down
#path = '/home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/yzAlign_run456195948_221112200957.pkl'
#path = '/home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/updownrun456195948_221112200957.pkl'


#first weekend files

path1 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run447162516_221106162754.pkl" #200 MeV,14.9Gb, threshold not mentioned Run 57
path2 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run445203522_221104212842.pkl" #120 MeV, 2.3Gb, threshold not mentioned Run 031, may be junk
path3 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run446172943_221105174846.pkl" #80 MeV,  2.2Gb, threshold not mentioned Run 38

#second weekend files

#100e
path4 = "/home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/run456195948_221112200957.pkl" #200MeV 100e
path5 = "/home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/run456092458_221112092919.pkl" #120 MeV 100e
path6 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456114514_221112120005.pkl" #80MeV 100e

#200e
#path4 = "/home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/run461175005_221114175310.pkl" #200MeV 200e
#path5 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456100239_221112100811.pkl" #120MeV 200e
#path6 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run461201255_221114201300.pkl" #80MeV 200e

#300e
#path7= "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/noisyLooking/run456083237_221112083509.pkl" #200MeV 300e (junk file)
#path8 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/noisyLooking/run456083236_221112083241.pkl" #200MeV 300e (junk file)
#path9 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456103757_221112104316.pkl" #120MeV 300e



events1 = []
events2 = []
events3 = []
events4 = []
events5 = []
events6 = []
events8 = []
events9 = []

allPaths=[path1, path2, path3, path4, path5, path6]#,, path8, path9]
allEvents=[events1, events2, events3, events4, events5, events6]#,, events8, events9]
allPaths=[path4]
nEvents = -1 #1e7


for pathNum, path in enumerate(allPaths):
    if nEvents > 0 :
        f = IntProgress(min=0, max=nEvents, description="Processing:")
        display(f)
    for i,event in enumerate(readEvents(path, nEvents=nEvents)):
        if nEvents > 0 and i>0 and i%1e4 == 0: f.value += 1e4
        EventStatus = True
        event.clusters = [cluster for cluster in event.clusters]
        if pathNum==1: #only for 6 alpide files
            for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_5"]: 
                clusters = event.selectDetector(alpide)
                if len(clusters) != 1:                                        #select for single cluster events that show up on all detectors only
                    EventStatus = False
                    break
            if EventStatus: 
                allEvents[pathNum].append(copy.deepcopy(event))
            
        else:
            
            for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]: 
                clusters = event.selectDetector(alpide)
                if len(clusters) != 1:                                        #select for single cluster events that show up on all detectors only
                    EventStatus = False
                    break
            if EventStatus: 
                allEvents[pathNum].append(copy.deepcopy(event))
    
    if 'f' in locals():
        f.bar_style = "success"
        print("In path "+str(pathNum+1)+" Found events:",len(allEvents[pathNum]),", Fraction:",len(allEvents[pathNum])*100/nEvents,"% in file "+path)

Going to process all events.
In path 1 Found events: 25380 , Fraction: -2538000.0 % in file /home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/run456195948_221112200957.pkl


# Flip

In [41]:
indexToBeFlipped = [0] #[3,4]
straight1=[]
straight2=[]
straight3=[]
straight4=[]
straight5=[]
straight6=[]
straight7=[]
straight8=[]

straightEvents = [straight1]#, straight2,straight3] #, straight4, straight5, straight6]#,, straight7, straight8]
for pathNum in range(len(straightEvents)):
    if pathNum in indexToBeFlipped:
        for event in allEvents[pathNum]:
            for cluster in event.clusters:
                cluster=cluster.flipYaxis()
            straightEvents[pathNum].append(event)
    else:
        straightEvents[pathNum]=allEvents[pathNum]


# Add Tracks

In [42]:
tracked1=[]
tracked2=[]
tracked3=[]
tracked4=[]
tracked5=[]
tracked6=[]
tracked7=[]
tracked8=[]

eventsTracked = [tracked1]#, tracked2, tracked3] #, tracked4, tracked5, tracked6]#,, tracked7, tracked8]

for pathNum, events in enumerate(straightEvents):
    for event in events:
        TrackLeft = Track()
        if pathNum==1:
            TrackLeft.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_3","ALPIDE_5"]])
        else:
            TrackLeft.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_3","ALPIDE_4"]])
        
        TrackRight = Track()
        TrackRight.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
        vertex = Vertex()
        vertex.fromTracks([TrackLeft,TrackRight])
        event.addTrack(TrackLeft)
        event.addTrack(TrackRight)
        event.vertex = vertex
        event.calculateMeanClusterSize()
        eventsTracked[pathNum].append(event)
    #allEvents[pathNum]=eventsTracked[pathNum]

# Plot

In [51]:
%jsroot on

#the file to be plotted
fileIndex=0

hVertexXY = ROOT.TH2F("VertexXY","VertexXY from File "+allPaths[fileIndex][-29:],300,-3.5,3.5,100,-3.5,3.5)
hVertexXY.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXY.GetYaxis().SetTitle("Vertex Y position [mm]")
hVertexXZ = ROOT.TH2F("VertexXZ","VertexXZ from File "+allPaths[fileIndex][-29:],300,-3.5, 3.5,100,-23,-16)
hVertexXZ.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXZ.GetYaxis().SetTitle("Vertex Z position [mm]")
hVertexYZ = ROOT.TH2F("VertexYZ","VertexYZ from File "+allPaths[fileIndex][-29:],300,-30,30,100,-30,30)
hVertexYZ.GetXaxis().SetTitle("Vertex Y position [mm]")
hVertexYZ.GetYaxis().SetTitle("Vertex Z position [mm]")


#first or second weekend because of target shift
weekend=0
if fileIndex<3:
    weekend=1
else:
    weekend=2
if (weekend==1):
    xMin=-1
    xMax=1.5

    yMin=-4.25
    yMax=3.75

    zMin=-19.5#75
    zMax=-19.4#25
    

elif (weekend==2):
    xMin=-1
    xMax=1.5

    yMin=-4
    yMax=4

    zMin=-18.5
    zMax=-18

    
volume= (xMax-xMin) * (yMax-yMin) * (zMax-zMin)
 
        
for event in eventsTracked[fileIndex]:
    #event.calculateMeanClusterSize()
   # if event.meanClusterSize > 30:
    #for i in range(round(event.meanClusterSize)):
    #if( xMin < event.vertex.point[0] < xMax and yMin < event.vertex.point[1] < yMax and zMin < event.vertex.point[2] < zMax):
    #if( zMin < event.vertex.point[2] < zMax):
    hVertexXY.Fill(event.vertex.point[0],event.vertex.point[1])
    hVertexXZ.Fill(event.vertex.point[0],event.vertex.point[2])
    hVertexYZ.Fill(event.vertex.point[1],event.vertex.point[2])
             
        
        
        
canvas = ROOT.TCanvas("","",1300,500)
canvas.Divide(3)

canvas.cd(1)
ROOT.gPad.SetGrid(2)
hVertexXY.Draw("COLZ") #surf2

canvas.cd(2)
ROOT.gPad.SetGrid(2)
hVertexXZ.Draw("COLZ")

canvas.cd(3)
ROOT.gPad.SetGrid(2)
hVertexYZ.Draw("COLZ")

canvas.Draw()

Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexXZ (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexYZ (Potential memory leak).


In [50]:
%jsroot on

hVertexXYs = [ROOT.TH2F("VertexXY","VertexXY from File "+allPaths[i][-29:],200,-10,10,200,-10,10) for i in range(1)]

for hVertexXY in hVertexXYs:
    hVertexXY.GetXaxis().SetTitle("Vertex X position [mm]")
    hVertexXY.GetYaxis().SetTitle("Vertex Y position [mm]")
#the file to be plotted
for fileIndex in range(1): 

    #first or second weekend because of target shift
    weekend=0
    if fileIndex<3:
        weekend=1
    else:
        weekend=2
    if (weekend==1):
        xMin=-1
        xMax=1.5

        yMin=-4.25
        yMax=3.75

        zMin=-19.5#75
        zMax=-19.4#25


    elif (weekend==2):
        xMin=-1
        xMax=1.5

        yMin=-4
        yMax=4

        zMin=-18.5
        zMax=-18


    volume= (xMax-xMin) * (yMax-yMin) * (zMax-zMin)


    for event in eventsTracked[fileIndex]:
        #event.calculateMeanClusterSize()
       # if event.meanClusterSize > 30:
        #for i in range(round(event.meanClusterSize)):
        #if( xMin < event.vertex.point[0] < xMax and yMin < event.vertex.point[1] < yMax and zMin < event.vertex.point[2] < zMax):
        if( zMin < event.vertex.point[2] < zMax):
        #if event.vertex.dca < 2:
            hVertexXYs[fileIndex].Fill(event.vertex.point[0],event.vertex.point[1])
             
        
        
        
canvas = ROOT.TCanvas("","",1200,900)
canvas.Divide(3,2)
for i in range(1):
    canvas.cd(i+1)
    ROOT.gPad.SetGrid(2)
    hVertexXYs[i].Draw("COLZ") #surf2

canvas.Draw()    
        


IndexError: list index out of range

Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).


In [52]:
%jsroot on

#somehow find ratio between the two weekends
#if entries overlap (both nonzero), then divide 
#BUT what are we doing the ratio between?


hVertexXYs = [ROOT.TH2F("VertexXY","VertexXY from File "+allPaths[i][-29:],200,-10,10,200,-10,10) for i in range(6)]

for hVertexXY in hVertexXYs:
    hVertexXY.GetXaxis().SetTitle("Vertex X position [mm]")
    hVertexXY.GetYaxis().SetTitle("Vertex Y position [mm]")
#the file to be plotted
for fileIndex in range(6): 

    #first or second weekend because of target shift
    weekend=0
    if fileIndex<3:
        weekend=1
    else:
        weekend=2
    if (weekend==1):
        xMin=-1
        xMax=1.5

        yMin=-4.25
        yMax=3.75

        zMin=-19.5#75
        zMax=-19.4#25


    elif (weekend==2):
        xMin=-1
        xMax=1.5

        yMin=-4
        yMax=4

        zMin=-18.5
        zMax=-18


    volume= (xMax-xMin) * (yMax-yMin) * (zMax-zMin)


    for event in eventsTracked[fileIndex]:
        #event.calculateMeanClusterSize()
       # if event.meanClusterSize > 30:
        #for i in range(round(event.meanClusterSize)):
        #if( xMin < event.vertex.point[0] < xMax and yMin < event.vertex.point[1] < yMax and zMin < event.vertex.point[2] < zMax):
        if( zMin < event.vertex.point[2] < zMax):
        #if event.vertex.dca < 2:
            hVertexXYs[fileIndex].Fill(event.vertex.point[0],event.vertex.point[1])
             
        
        
        
canvas = ROOT.TCanvas("","",1200,900)
canvas.Divide(3,2)
for i in range(6):
    canvas.cd(i+1)
    ROOT.gPad.SetGrid(2)
    hVertexXYs[i].Draw("COLZ") #surf2

canvas.Draw()    
        


IndexError: list index out of range

Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).


## Openning Angle

In [65]:
%jsroot on

detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]
canvas = ROOT.TCanvas("","",2400,900) #1200 looks nicer but 2400 matches tobis plots
canvas.Divide(3,2)
legend = ROOT.TLegend()


xTMin=-2
xTMax=2

yTMin=-5
yTMax=5

zTMin=-22 #bigger to accomodate targets within shift
zTMax=-17

tVolume= (xTMax-xTMin) * (yTMax-yTMin) * (zTMax-zTMin)

xAMin=4.5
xAMax=8

yAMin=-4
yAMax=4

zAMin=-20
zAMax=-17.5

aVolume= (xAMax-xAMin) * (yAMax-yMin) * (zAMax-zAMin)

xMin=-1
xMax=1.5

yMin=-4
yMax=4

zMin=-18.5
zMax=-18

volume= (xMax-xMin) * (yMax-yMin) * (zMax-zMin)

# Calculate histogram

hOpenningAngle  = ROOT.TH1F("Openning Angle", "Opening Angle Target Region First Week 200 MeV", 720,0,180)
hOpenningAngle2 = ROOT.TH1F("Openning Angle", "Opening Angle Target Region First Week 120 MeV", 720,0,180) 
hOpenningAngle3 = ROOT.TH1F("Openning Angle", "Opening Angle Target Region First Week 80 MeV", 180,0,180)
hOpenningAngle4 = ROOT.TH1F("Openning Angle", "Opening Angle Target Region Second Week 200 MeV", 30,60,120)
hOpenningAngle5 = ROOT.TH1F("Openning Angle", "Opening Angle Target Region Second Week 120 MeV", 30,60,120) 
hOpenningAngle6 = ROOT.TH1F("Openning Angle", "Opening Angle Target Region Second Week 80 MeV", 30,60,120)

eLabel=["200 MeV", "120 MeV", "80 MeV"]

hDivHists= [ROOT.TH1F("Openning Angle Divided", "Opening Angle First Week Divided By Second Week "+eLabel[i], 30,60,120) for i in range(3)]

for hDivHist in hDivHists:
    hDivHist.GetXaxis().SetTitle("Openning Angle [^{o}]")

histograms = [hOpenningAngle, hOpenningAngle2, hOpenningAngle3,hOpenningAngle4, hOpenningAngle5, hOpenningAngle6]


for index, eventSet in enumerate(eventsTracked):
    if index < 6:
        for event in eventSet: 
            #if( xAMin < event.vertex.point[0] < xAMax and yAMin < event.vertex.point[1] < yAMax and zAMin < event.vertex.point[2] < zAMax):
            histograms[1].Fill(event.vertex.openingAngle*360/(2*math.pi),1/tVolume)
            if( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                histograms[index].Fill(event.vertex.openingAngle*360/(2*math.pi),1/tVolume)

               
                
#Draw histogram
colors = [1,2,3,4,6,7,8,9]
labels=["200MeV Week 1", "120MeV Week 1", "80MeV Week 1","200MeV Week 2", "120MeV Week 2", "80MeV Week 2"]
gauss1 = ROOT.TF1("gauss", "gaus")
gauss2 = ROOT.TF1("gauss", "gaus")
gauss3 = ROOT.TF1("gauss", "gaus")
gauss4 = ROOT.TF1("gauss", "gaus")
gauss5 = ROOT.TF1("gauss", "gaus")
gauss6 = ROOT.TF1("gauss", "gaus")




gauss= [gauss1,gauss2, gauss3,gauss4,gauss5, gauss6]
lines = []
#200MeV, 120MeV, 80MeV

ppOpAng=[87.08, 88.2, 88.76]
p2pOpAng=[82.56, 79.82, 74.78]

for row in range(1):
    for index,chist in enumerate(histograms[0:2]):
    #for index,chist in enumerate(hDivHists):
        canRow=0
        if row==1:
            canRow+=3
        
        pad=canvas.cd(index+1+canRow)
        
        if row==1:
            pad.SetLogy()
        
        #chist.Divide(histograms[index],histograms[index+3])
        #print(chist)
        #gauss[index].SetLineColor(colors[index])
        chist.Scale(1/chist.Integral())
        chist.SetMaximum(1)

        #chist.SetBinErrorOption(ROOT.TH1.kNoError)
        #chist.Fit(gauss[index])

        # retrieve the fit parameters
        #mean = gauss[index].GetParameter(1)
        #sigma = gauss[index].GetParameter(2)
        #sigNormd = sigma/np.sqrt(chist.GetEntries())

        #chist.SetLineColor(colors[index])
        #chist.SetMarkerColor(colors[index])    
        chist.SetMarkerStyle(20)
        chist.SetLineWidth(1)
        chist.SetXTitle("Opening Angle / ^{o} ")
        chist.SetYTitle("Cluster Counts / Normalized to Area") 
        #legend.AddEntry(chist,labels[index]+ " #mu= " + str(mean)[0:5]) + " and #sigma/#sqrtN= " + str(sigNormd)[0:5])
        legend.AddEntry(chist, labels[index]+ ", #mu= " + str(chist.GetMean())[0:5])    
        chist.Draw("P")

        #line1 = ROOT.TLine(p2pOpAng[index], 0, p2pOpAng[index], chist.GetMaximum())
        #line1.SetLineColor(5)
        #line1.SetLineWidth(2)
        #line1.Draw()
        #lines.append(line1)

        line2 = ROOT.TLine(ppOpAng[index], 0, ppOpAng[index], chist.GetMaximum())
        line2.SetLineColor(6)
        line2.SetLineWidth(2)
        line2.SetLineStyle(2)
        line2.Draw()
        lines.append(line2)
        pad.Update()
    
#legend.Draw() 
canvas.Update()
canvas.Draw()
#canvas.SetGridx(2)

Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle Divided (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle Divided (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle Divided (Potential memory leak).


In [ ]:
%jsroot on

canvas = ROOT.TCanvas("","",1200,450)
canvas.Divide(3)
legend = ROOT.TLegend()

# Select alignment
ALPIDES = ['ALPIDE_0','ALPIDE_1', 'ALPIDE_2', 'ALPIDE_3', 'ALPIDE_4'] 
null_displacement={
     'ALPIDE_0': [0, 0, 0], 
     'ALPIDE_1': [0, 0, 0], 
     'ALPIDE_2': [0, 0, 0], 
     'ALPIDE_3': [0, 0, 0], 
     'ALPIDE_4': [0, 0, 0]
}
custom_displacement={
     'ALPIDE_0': [0, 0, 0], 
     'ALPIDE_1': [1.1, 0, 0], 
     'ALPIDE_2': [0.2, 0, 0], 
     'ALPIDE_3': [0, 0, 0], 
     'ALPIDE_4': [0, 0, 0]
}

alignment01092023={
     'ALPIDE_0': [-0.28125, -0.225, 0.15], 
     'ALPIDE_1': [-0.225, -0.24375, 0.15], 
     'ALPIDE_2': [0.3, -0.24375, 0.075], 
     'ALPIDE_3': [-0.3, -0.2625, 0.075], 
     'ALPIDE_4': [-0.3, -0.24375, 0.15]
}

best_displacement_no_multiple_scat ={
    'ALPIDE_0': [0.5, -0.0546875, 0], 
    'ALPIDE_1': [0.65, -0.0546875, 0], 
    'ALPIDE_2': [0.75, -0.06875, 0], 
    'ALPIDE_3': [0.2, -0.125, 0], 
    'ALPIDE_4': [0.5546875, -0.05, 0]
}

berkin_ml_displacement={
    'ALPIDE_0': [-0.20006977021694183, -0.8966014385223389,0], 
    'ALPIDE_1': [0.7433865666389465, -0.8977992534637451,0], 
    'ALPIDE_2': [-0.8155627250671387, -0.9625105261802673,0], 
    'ALPIDE_3': [0.10266361385583878, 0.31308597326278687,0], 
    'ALPIDE_4': [0.4219255745410919, 1.322750449180603,0]
}
temp_x_align={
    'ALPIDE_0': [0, -1.75, 0], 
    'ALPIDE_1': [1.1, 1.296875, 0], 
    'ALPIDE_2': [0.2, 1.0, 0], 
    'ALPIDE_3': [0, 1.75, 0], 
    'ALPIDE_4': [0, -1.25, 0]
}
displacement=custom_displacement
# Calculate histogram

hOpenningAngle  = ROOT.TH2F("Opening Angle", "Target Region First Week 200 MeV", 100, -8, 8, 200, 70, 100)
hOpenningAngle2 = ROOT.TH2F("Opening Angle", "Target Region First Week 120 MeV", 100,-8,8, 200, 70, 100) 
hOpenningAngle3 = ROOT.TH2F("Opening Angle", "Target Region First Week 80 MeV", 100,-8,8, 200, 70, 100)

eLabel=["200 MeV", "120 MeV", "80 MeV"]

hDivHists= [ROOT.TH1F("Openning Angle Divided", "Opening Angle First Week Divided By Second Week "+eLabel[i], 30,60,120) for i in range(3)]

for hDivHist in hDivHists:
    hDivHist.GetXaxis().SetTitle("Openning Angle [^{o}]")

histograms = [hOpenningAngle, hOpenningAngle2, hOpenningAngle3] #,hOpenningAngle4, hOpenningAngle5, hOpenningAngle6]

dcaVals = []
#trackRMSVals = []
tempEvents=copy.deepcopy(eventsTracked) #problem is that the events are changed in the algorithm so a copy must be used loadEvents("/home/cehrich/Software/uits3_krakow22/cleanEvents_brute.pkl")

#for set in tempEvents:
    #print("set", set)
for event in tempEvents[0]: #set:    
    #print("event.selectDetector(alpide)[0]", event.selectDetector(alpide)[0])
    TrackLeft = Track()
    TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:3]) for alpide in ['ALPIDE_3','ALPIDE_4']])
    #why does this line give me a list index out of range error? 
    
    TrackRight = Track()
    TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:3]) for alpide in ['ALPIDE_0','ALPIDE_1','ALPIDE_2']]) 
    
    vertex = Vertex()
    vertex.fromTracks([TrackLeft,TrackRight])
    for alpide in ALPIDES:
        event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:5])
    event.addTrack(TrackLeft)
    event.addTrack(TrackRight)
    event.vertex = vertex
    dcaVals.append(vertex.dca)
    #trackRMSVals.append(event.trackRMS)
 
for index, eventSet in enumerate(tempEvents):
    if index < 3:
        for event in eventSet: 
            if( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                histograms[index].Fill(event.selectDetector("ALPIDE_1")[0].localPos[1], event.vertex.openingAngle*360/(2*math.pi)) 
               
                
#Draw histogram
colors = [1,2,3,4,6,7,8,9]
labels=["200MeV Week 1", "120MeV Week 1", "80MeV Week 1","200MeV Week 2", "120MeV Week 2", "80MeV Week 2"]

lines = []
#200MeV, 120MeV, 80MeV

ppOpAng=[87.08, 88.2, 88.76]
p2pOpAng=[82.56, 79.82, 74.78]



xTMin=-2
xTMax=2

zTMin=-22
zTMax=-17



for row in range(1):
    for index,chist in enumerate(histograms[0:3]):
        canRow=0
        if row==1:
            canRow+=3
        
        pad=canvas.cd(index+1+canRow)
        
        if row==1:
            pad.SetLogy()
        

        chist.SetLineColor(colors[index])

        chist.SetLineWidth(1)
        chist.SetYTitle("Opening Angle / ^{o} ")
        chist.SetXTitle("Local Y Position of ALPIDE 1 Cluster / [mm]") 
        legend.AddEntry(chist, labels[index]+ ", #mu= " + str(chist.GetMean())[0:5])
        chist.Draw("COLZ")

        #line1 = ROOT.TLine(-7, p2pOpAng[index], 7, p2pOpAng[index])
        #line1.SetLineColor(6)
        #line1.SetLineWidth(2)
        #line1.SetLineStyle(2)
        #line1.Draw()
        #lines.append(line1)

        line2 = ROOT.TLine(-7, ppOpAng[index], 7, ppOpAng[index])
        line2.SetLineColor(7)
        line2.SetLineWidth(2)
        line2.SetLineStyle(2)
        line2.Draw()
        lines.append(line2)
        pad.Update()
    
canvas.Update()
canvas.Draw()

Warning in <TROOT::Append>: Replacing existing TH1: Opening Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Opening Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Opening Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle Divided (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle Divided (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle Divided (Potential memory leak).


# Plot Cluster Sizes

In [ ]:
%jsroot 

detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]
colors = [1,2,3,4,6,7,8,9]
canvas = ROOT.TCanvas("","",600,600)
legend = ROOT.TLegend()
det=0
# Calculate histogram
hClusSize = [ROOT.TH1F("Cluster_Size", "ALPIDE "+str(det)+" at 80 MeV Threshold  100e", 41, -0.5, 40.5) for detr in detectors]
hClusSize1 = [ROOT.TH1F("Cluster_Size", "Vertex Region Cluster Sizes from File "+path2[-10:], 41, -0.5, 40.5) for detr in detectors]

histograms = [hClusSize, hClusSize2]

AluRegion = True
AroundTargetRegion= True

if AluRegion:
    xMin=4.5
    xMax=8

    yMin=-4
    yMax=4

    zMin=-20
    zMax=-17.5
    

elif AroundTargetRegion:
    xMin=-4.5
    xMax=4.5

    yMin=-6
    yMax=6

    zMin=-23
    zMax=-15

xTMin=-2
xTMax=2

yTMin=-5
yTMax=5

zTMin=-20
zTMax=-17

volume= (xMax-xMin) * (yMax-yMin) * (zMax-zMin)
tVolume= (xTMax-xTMin) * (yTMax-yTMin) * (zTMax-zTMin)


for event in eventsTracked[5]:#eventSet:
    for index,detector in enumerate(detectors): #index is the dectector
        for cluster in event.clusters:
            if cluster.detector == detector:
                if( xMin < event.vertex.point[0] < xMax and yMin < event.vertex.point[1] < yMax and zMin < event.vertex.point[2] < zMax):
                    if not( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                        if event.vertex.point[0] > 0:
                            histograms[0][index].Fill(cluster.size, 2/volume)
                        else:
                            histograms[1][index].Fill(cluster.size, 2/volume)   
    
    
                    
#Draw histogram
labels=["x>0","x<0"]

#fits
gauss1 = ROOT.TF1("gauss", "gaus")
gauss2 = ROOT.TF1("gauss", "gaus")

gauss= [gauss1,gauss2, gauss3, gauss4, gauss5, gauss6, gauss7, gauss8]
canvas.Clear()
for index,hist in enumerate(histograms):
    #gauss[index].SetLineColor(colors[index])
    hist[det].Scale(1/hist[det].Integral())
    hist[det].SetMaximum(0.3)
    hist[det].Draw("Same P  L")
    #hist[det].Fit(gauss[index])

    # retrieve the fit parameters
    #mean = gauss[index].GetParameter(1)
    #sigma = gauss[index].GetParameter(2)
    #sigNormd = sigma/np.sqrt(chist.GetEntries())

    hist[det].SetLineColor(colors[index])
    hist[det].SetMarkerColor(colors[index])
    hist[det].SetMarkerStyle(20)
    hist[det].SetLineWidth(3)
    hist[det].SetXTitle("Cluster size [pixels]")
    hist[det].SetYTitle("Cluster Counts / Normalized to Area")
    #legend.AddEntry(hist[det], labels[index]+ " #mu= " + str(mean)[0:5] + " and #sigma/#sqrtN= " + str(sigNormd)[0:5]) 
    legend.AddEntry(hist[det], labels[index]+ ", #mu= " + str(hist[det].GetMean())[0:5])    
    
    
canvas.Update()
legend.Draw() 
canvas.Draw()

NameError: name 'hClusSize2' is not defined

Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).


In [ ]:
%jsroot 

detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]
colors = [1,2,3,4,6,7,8,9]
canvas = ROOT.TCanvas("","",600,600)
legend = ROOT.TLegend()
det=0
# Calculate histogram

title1="Aluminum Region Second Weekend ALPIDE "+str(det) #+" for all Thresholds and Energies"
title2="x < 0 ALPIDE "+str(det)+" at Threshold  100e for Energies  200MeV, 120MeV, and 80MeV "
hClusSize  = [ROOT.TH1F("Cluster_Size", title1, 20, -0.5, 40.5) for detr in detectors]
hClusSize2 = [ROOT.TH1F("Cluster_Size", "Vertex Region Cluster Sizes from File "+path2[-10:], 20, -0.5, 40.5) for detr in detectors]
hClusSize3 = [ROOT.TH1F("Cluster_Size", "Vertex Region Cluster Sizes from File "+path3[-10:], 20, -0.5, 40.5) for detr in detectors]

hClusSize4 = [ROOT.TH1F("Cluster_Size", title1, 20, -0.5, 40.5) for detr in detectors]
hClusSize5 = [ROOT.TH1F("Cluster_Size", "Vertex Region Cluster Sizes from File "+path2[-10:], 20, -0.5, 40.5) for detr in detectors]
hClusSize6 = [ROOT.TH1F("Cluster_Size", "Vertex Region Cluster Sizes from File "+path3[-10:], 20, -0.5, 40.5) for detr in detectors]
hClusSize7 = [ROOT.TH1F("Cluster_Size", "Vertex Region Cluster Sizes from File "+path3[-10:], 20, -0.5, 40.5) for detr in detectors]

histograms = [hClusSize, hClusSize2, hClusSize3, hClusSize4, hClusSize5, hClusSize6, hClusSize6, hClusSize7]

AluRegion = True
TargetRegion=False

if AluRegion:
    xMin=4.5
    xMax=8

    yMin=-4
    yMax=4

    zMin=-20
    zMax=-17.5
    

elif TargetRegion:
    xMin=-4.5
    xMax=4.5

    yMin=-6
    yMax=6

    zMin=-23
    zMax=-15

xTMin=-2
xTMax=2

yTMin=-5
yTMax=5

zTMin=-20
zTMax=-17

volume= (xMax-xMin) * (yMax-yMin) * (zMax-zMin)
tVolume= (xTMax-xTMin) * (yTMax-yTMin) * (zTMax-zTMin)

#eventArr = [eventsUncut, eventsUncut2, eventsUncut3]

for hist, eventSet in enumerate(eventsTracked):
    for event in eventSet:
        for index,detector in enumerate(detectors): #index is the dectector
            for cluster in event.clusters:
                if cluster.detector == detector:
                    if( xMin < event.vertex.point[0] < xMax and yMin < event.vertex.point[1] < yMax and zMin < event.vertex.point[2] < zMax):
                    #if( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                     #       if event.vertex.point[0] < 0:
                        histograms[hist][index].Fill(cluster.size, 1/volume)
                    #if not( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                     #   histograms[hist][index].Fill(cluster.size, 1/tVolume)
    
    
                    
#Draw histogram
labels=["200MeV", "120MeV", "  80MeV","200e, 200MeV", "200e, 120MeV", "200e, 80MeV","300e, 200MeV", "300e, 120MeV"]


#fits
gauss1 = ROOT.TF1("gauss", "gaus")
gauss2 = ROOT.TF1("gauss", "gaus")
gauss3 = ROOT.TF1("gauss", "gaus")
gauss4 = ROOT.TF1("gauss", "gaus")
gauss5 = ROOT.TF1("gauss", "gaus")
gauss6 = ROOT.TF1("gauss", "gaus")
gauss7 = ROOT.TF1("gauss", "gaus")
gauss8 = ROOT.TF1("gauss", "gaus")
#gauss1.SetLineColor(ROOT.kBlack)
#gauss2.SetLineColor(ROOT.kRed)
#gauss3.SetLineColor(ROOT.kGreen)
gauss= [gauss1,gauss2, gauss3, gauss4, gauss5, gauss6, gauss7, gauss8]

for index,hist in enumerate(histograms[0:3]):
    #gauss[index].SetLineColor(colors[index])
    hist[det].Scale(1/hist[det].Integral())
    hist[det].SetMaximum(0.4)
    hist[det].Draw("Same PL  ")
    #hist[det].Fit(gauss[index])

    # retrieve the fit parameters
    #mean = gauss[index].GetParameter(1)
    #sigma = gauss[index].GetParameter(2)
    #sigNormd = sigma/np.sqrt(chist.GetEntries())

    hist[det].SetLineColor(colors[index])
    hist[det].SetMarkerColor(colors[index])
    hist[det].SetMarkerStyle(20)
    hist[det].SetLineWidth(3)
    hist[det].SetXTitle("Cluster size [pixels]")
    hist[det].SetYTitle("Cluster Counts / Normalized to Area")
    #print(hist[det].GetEntries())
    #legend.AddEntry(hist[det], labels[index]+ " #mu= " + str(mean)[0:5] + " and #sigma/#sqrtN= " + str(sigNormd)[0:5]) 
    legend.AddEntry(hist[det], labels[index]+ ", #mu= " + str(hist[det].GetMean())[0:5])    
    #else:
      #  legend.AddEntry(hist[det],"Target Region " + labels[index])
    
    
    
legend.Draw() 
canvas.Draw()

NameError: name 'eventsTracked' is not defined

Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in

In [ ]:
%jsroot on
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4","ALPIDE_5"]
colors = [1,2,3,4,6,7]
canvas = ROOT.TCanvas("","",600,600)
legend = ROOT.TLegend()

# Calculate histogram
hClusSize = [ROOT.TH1F("Cluster_Size", "Vertex Region with Background Subtracted, Cluster Sizes from File "+path2[-10:], 41, -0.5, 40.5) for det in detectors]
hClusSize2 = [ROOT.TH1F("Cluster_Size", "Cluster Sizes in Background Region from File "+path[-10:], 41, -0.5, 40.5) for det in detectors]

for event in eventsTracked[0]: #readEvents(path,1e5): #from path to FlippedEvents
    for cluster in event.clusters:
        #if cluster.detector == detector:
            #if( -2 < event.vertex.point[0] < 2 and -20 < event.vertex.point[2] < -17 and -5 < event.vertex.point[1] < 5):
        hClusSize[int(cluster.detector[-1])].Fill(cluster.size)
            #elif ( -4.5 < event.vertex.point[0] < 4.5 and -23 < event.vertex.point[2] < -15 and -6 < event.vertex.point[1] < 6):
            #    hClusSize2[index].Fill(cluster.size, 5/31)
#Draw histogram
#for index,hist in enumerate(hClusSize):
#index = 4
for index,hist in enumerate(hClusSize[0:6]):
    #hist.Scale(1/hist.Integral())
    #hist.SetMaximum(370)
    #hist = hClusSize[index]
    hist.Draw("Same P L ")
    hist.SetLineColor(colors[index])
    hist.SetMarkerColor(colors[index])
    hist.SetMarkerStyle(20)
    hist.SetLineWidth(3)
    hist.SetXTitle("Cluster size [pixels]")
    hist.SetYTitle("Normalized Counts") # (normalized to integral)")
    legend.AddEntry(hist,"Vertex " + detectors[index])
    #ROOT.gPad.SetLogy()

if False:    
    for index,hist in enumerate(hClusSize2[0:1]):
        hist.Scale(1/hist.Integral())
            #hist.SetMaximum(0.2)
        hist.Draw("Same P L")
        hist.SetLineColor(colors[index])
        hist.SetMarkerColor(colors[index])
        hist.SetMarkerStyle(2)
        hist.SetLineWidth(3)
        hist.SetLineStyle(2)
        hist.SetXTitle("Cluster size [pixels]")
        hist.SetYTitle("Normalized Counts") # (normalized to integral)")
        legend.AddEntry(hist,"Background " + detectors[index])
            #ROOT.gPad.SetLogy()
    
legend.Draw() 
canvas.Draw()

In [ ]:
%jsroot on
# plot RMS of distances between clusters and the track
hRMS = ROOT.TH1F("RMS cluster","RMS Cluster",22*5,-0.1,2.1)
hRMS.GetXaxis().SetTitle("RMS(cluster distance in track) [mm]")
hRMS.GetYaxis().SetTitle("Count")

for event in eventsTracked[0]:
    hRMS.Fill(event.tracks[1].rms)


canvas = ROOT.TCanvas()
canvas.SetGridx(2)
hRMS.Draw("")
canvas.Draw()

# Plot Correlation

In [ ]:
%jsroot on

nBins = {"x-Axis" : 300, "y-Axis" : 150}
nRange = {"x-Axis" : 15, "y-Axis" : 7.5}
axis = {"x-Axis":0, "y-Axis":1}

detector1 = "ALPIDE_0"
detector2 = "ALPIDE_4"
axis1="x-Axis"
axis2="y-Axis"
lowerClustSizThresh = 7  #thresholds for how big to make the file for analysis
upperClustSizThresh = 15
clustNumThresh = 1
    
hCorr = ROOT.TH2F("Corr_"+detector1+"_"+axis1+"_"+detector2+"_"+axis2,"Correlation", nBins.get(axis1),-nRange.get(axis1),nRange.get(axis1),nBins.get(axis2),-nRange.get(axis2),nRange.get(axis2))
hCorr.SetXTitle(detector1 + " - " + axis1)
hCorr.SetYTitle(detector2 + " - " + axis2)

for event in eventsTracked[0]: #readEvents(path2, nEvents=1e5):
    clustersDetector1 = event.selectDetector(detector1)
    clustersDetector2 = event.selectDetector(detector2)
    #selectedClusters1 = [cluster for cluster in clustersDetector1 if upperClustSizThresh > cluster.size > lowerClustSizThresh]
    #selectedClusters2 = [cluster for cluster in clustersDetector2 if upperClustSizThresh > cluster.size > lowerClustSizThresh]
    #if len(selectedClusters1) <= clustNumThresh and len(selectedClusters2) <= clustNumThresh:
    for cluster1 in clustersDetector1:
        for cluster2 in clustersDetector2:
            hCorr.Fill(cluster1.localPos[axis.get(axis1)],cluster2.localPos[axis.get(axis2)])
                

canvas = ROOT.TCanvas()
hCorr.Draw("COLZ")
canvas.Draw()

In [ ]:
%jsroot on

nBins = {"x-Axis" : 500, "y-Axis" : 500}
nRange = {"x-Axis" : 15, "y-Axis" : 7.5}
axis = {"x-Axis":0, "y-Axis":1}
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4","ALPIDE_5"]

detector1 = "ALPIDE_3"
detector2 = "ALPIDE_4"
#ALPIDE 0 vs ALPIDE 4 
hClusSize = [ROOT.TH2F("Cluster_Size", "Cluster Sizes Vertex Region Cluster Sizes from File "+path2[-10:], 41, -0.5, 40.5,41, -0.5, 40.5) for det in detectors]

for event in eventsTracked[0]: #readEvents(path, nEvents=-1):
    clustersDetector1 = event.selectDetector(detector1)
    clustersDetector2 = event.selectDetector(detector2)
    for cluster1 in clustersDetector1:
        for cluster2 in clustersDetector2:
            hClusSize[0].Fill(cluster1.size,cluster2.size)
                

canvas = ROOT.TCanvas()
#hClusSize[0].Scale(1/hClusSize[0].Integral())
#hclusSize[0].SetMaximum(0.2)
hClusSize[0].Draw("COLZ")
#hClusSize[0].SetLineColor(colors[2])
#hClusSize[0].SetMarkerColor(colors[2])
#hClusSize[0].SetMarkerStyle(20)
#hClusSize[0].SetLineWidth(3)
#hClusSize[0].SetLineStyle(20)
hClusSize[0].SetXTitle("Cluster size ALPIDE 0 / [pixels]")
hClusSize[0].SetYTitle("Cluster size ALPIDE 4 / [pixels]") # (normalized to integral)")
legend.AddEntry(hClusSize[0],"Background " + detectors[index])
canvas.Draw()

## Plot Beam Energy vs Cluster Count vs Mean Free Path

In [ ]:
#1) Define geometry of the cyls the track passes through
#2) Get the cylinder normal vector (imagine a line parallel to a radius of the beam axis or global z)
#3) Check the angle at which the track passes through
#4) take difference between cyl normal vector and track

#from angle we can calculate the path through the Si and materialPath=25micron/cos(this angle)

hClusPath = ROOT.TH1F("Cluster_Path", "Mean Path Length through Detector Seen by Tracked Particles ", 750, -0, 100)

save = False #for saving image


    #define plot of ALPIDEs
fig = plt.figure(figsize=(3, 3), dpi=300)
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
#ax.set_proj_type('ortho')
ax.view_init(-75,30,60)

    
    #below follow ALPIDE cyl geometry def
    
ALPIDE_0 = Cylinder([0,0,12.5-7.5], [0,0,15], 30)
ALPIDE_1 = Cylinder([0,0,6.25-7.5], [0,0,15], 24)
ALPIDE_2 = Cylinder([0,0,0-7.5], [0,0,15], 18)

    #end cyl geometry def

ALPIDE_2.plot_3d(ax, alpha=1)
ALPIDE_1.plot_3d(ax, alpha=1)
ALPIDE_0.plot_3d(ax, alpha=1)

viewingAngles = [[0,0],[0,-90],[90,-90],[30,-55]]

meanPathLength = [] 




alpidesLeft = ["ALPIDE_3","ALPIDE_4"]
#alpidesRight = [ALPIDE_0,ALPIDE_1,ALPIDE_2]


if save: #saves figure
    for [angle1,angle2] in viewingAngles:
        ax.view_init(angle1, angle2)
        plt.savefig("targets"+str(angle1)+"_"+str(angle2)+".png",bbox_inches='tight')
        
#now we look for the intersect between the track and the cylinder
#approach: each intersection is at a certain radius along the global z axis. r=sqrt(x²+y²) and there is one intersect point/alpide at the radius...
#better approach:
#1) get slope of track #done in track.py
#2) define unit vector for this slope 
#Track.calcUnitVector
#3) loop over each ALPIDE: 
#      3a) define unit normal vector for the cluster coordinates
#      3b) angle=cos⁻¹(dot product of unit vectors)
#      3c) pathlength= thickness/cos(angle) = thickness/(dot product of unit vectors) %ignore 3b and do this


canvas = ROOT.TCanvas()
for event in eventsTracked[2]:
    pathLenSum=0
    numClusters=0
    for index, cluster in enumerate(event.clusters):
        trackSide=0
        numClusters+=1
        if (cluster.detector in alpidesLeft): #remember we are defining a normal vector for the cylinder. here we get the detector and have a radius
            trackSide=1
        
        normalVector=[cluster.globalPos[0],cluster.globalPos[1],0 ] #cluster coords perpendicular to z
        normalVector/=np.linalg.norm(normalVector)
        trackVector=event.tracks[trackSide].vector/np.linalg.norm(event.tracks[trackSide].vector) #np.linalg.norm should return the magnitude
        pathLength=0.025/np.dot(normalVector,trackVector)*1000 #get hypotenus (track length through det)convert mm to micron
        pathLenSum+=np.abs(pathLength)
    #draw a line form 0,0,globalPos[2] to cluster.globalPos and, then divide by magnitude of this new vector
    #mFP=(outerRadius-pathLenSum)/3 #this will be almost the same value always
    #meanPathLength[index]=pathLenSum/3
    hClusPath.Fill(pathLenSum/numClusters)
hClusPath.SetXTitle("Mean Track Length through ALPIDE's / Micron")
hClusPath.SetYTitle("Counts")
hClusPath.Draw("COLZ")
canvas.Draw()
#4) meanFreePath = {distanceToOuterCyl-(sum of path lengths)}/3
#5) add this MFP attribute to each track       

In [ ]:
%jsroot on
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4","ALPIDE_5"]
colors = [2,3,4,6,7,8,9]
canvas = ROOT.TCanvas("","",600,600)
legend = ROOT.TLegend()

# Calculate histogram
det=2

hCluster1 = ROOT.TH3F("Cluster_Size", "Beam Energy vs. Cluster Size vs. Track Length through ALPIDE "+str(det), 28, 70, 210, 41, -0.5, 40.5, 200, 20, 100)
#hCluster2 = ROOT.TH3F("Cluster_Size", "Beam Energy vs. Cluster Size vs. Track Length through ALPIDE "+str(det), 28, 70, 210, 41, -0.5, 40.5, 200, 20, 100)
#hCluster3 = ROOT.TH3F("Cluster_Size", "Beam Energy vs. Cluster Size vs. Track Length through ALPIDE "+str(det), 28, 70, 210, 41, -0.5, 40.5, 200, 20, 100)

#histograms = [hCluster1, hCluster2, hCluster3] #three beam energies

xMin=4.5
xMax=8

yMin=-4
yMax=4

zMin=-20
zMax=-17.5

xTMin=-2
xTMax=2

yTMin=-5
yTMax=5

zTMin=-20
zTMax=-17

volume= (xMax-xMin) * (yMax-yMin) * (zMax-zMin)
tVolume= (xTMax-xTMin) * (yTMax-yTMin) * (zTMax-zTMin)

energies=[200, 120, 80]
for hist, eventSet in enumerate(eventsTracked):
    if hist < 3:
        for event in eventSet:
            pathLenSum=0
            numClusters=0    
            for cluster in event.clusters:
                trackSide=0
                numClusters+=1
                if (cluster.detector in alpidesLeft): #remember we are defining a normal vector for the cylinder. here we get the detector and have a radius
                    trackSide=1

                normalVector=[cluster.globalPos[0],cluster.globalPos[1],0 ] #cluster coords perpendicular to z
                normalVector/=np.linalg.norm(normalVector)
                trackVector=event.tracks[trackSide].vector/np.linalg.norm(event.tracks[trackSide].vector) #np.linalg.norm should return the magnitude
                pathLength=0.025/np.dot(normalVector,trackVector)*1000 #get hypotenus (track length through det)convert mm to micron
                pathLenSum+=np.abs(pathLength)
            #for cluster in event.clusters:    
                hCluster1.Fill(energies[hist], cluster.size, np.abs(pathLength)) #,cluster.size, 1/tVolume)
    
    
                    
#Draw histogram
labels=["200MeV", "120MeV", "80MeV","200e, 200MeV", "200e, 120MeV", "200e, 80MeV","300e, 200MeV", "300e, 120MeV"]


#fits
gauss1 = ROOT.TF1("gauss", "gaus")
gauss2 = ROOT.TF1("gauss", "gaus")
gauss3 = ROOT.TF1("gauss", "gaus")


gauss= [gauss1,gauss2, gauss3]

#for index,hist in enumerate(histograms):
    #gauss[index].SetLineColor(colors[index])
    #hist[det].Fit(gauss[index])

    # retrieve the fit parameters
    #mean = gauss[index].GetParameter(1)
    #sigma = gauss[index].GetParameter(2)

    #hist[det].Scale(1/hist[det].Integral())
    #hist[det].SetMaximum(0.3)
    
    #hist[det].SetLineColor(colors[index])
ROOT.gPad.SetGrid(2)
#hCluster1[det].SetMarkerColor(colors[index])
#hCluster1[det].SetMarkerStyle(20)
#legend.AddEntry(hCluster1[det], labels[index])#+ " #mu " + str(mean)[0:3] + " and #sigma " + str(sigma)[0:3]) 


hCluster1.Draw("BOX3")
hCluster1.SetXTitle("Beam Energy / MeV")
hCluster1.SetYTitle("Cluster Size / [pixels]")
hCluster1.SetZTitle("Track Length through ALPIDE "+str(det)+" / Micron")



canvas.Draw()

In [ ]:
%jsroot on
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4","ALPIDE_5"]
colors = [2,3,4,6,7,8,9]
canvas = ROOT.TCanvas("","",600,600)
legend = ROOT.TLegend()

# Calculate histogram
det=0

hCluster1 = ROOT.TH2F("Cluster_Size", "Beam Energy 80 MeV through ALPIDE "+str(det),41, -0.5, 40.5, 200, 25, 55) 
hCluster2 = ROOT.TH2F("Cluster_Size", "Beam Energy 120 MeV through ALPIDE "+str(det),41, -0.5, 40.5, 200, 25, 55) 
hCluster3 = ROOT.TH2F("Cluster_Size", "Beam Energy 200 MeV through ALPIDE "+str(det), 41, -0.5, 40.5, 200, 25, 55) 

histograms = [hCluster1, hCluster2, hCluster3] #three beam energies

xMin=4.5
xMax=8

yMin=-4
yMax=4

zMin=-20
zMax=-17.5

xTMin=-2
xTMax=2

yTMin=-5
yTMax=5

zTMin=-20
zTMax=-17

volume= (xMax-xMin) * (yMax-yMin) * (zMax-zMin)
tVolume= (xTMax-xTMin) * (yTMax-yTMin) * (zTMax-zTMin)

energies=[200, 120, 80]
for hist, eventSet in enumerate(eventsTracked):
    if hist < 3:
        for event in eventSet:
            #pathLenSum=0
            numClusters=0    
            for cluster in event.clusters:
                #if( xMin < event.vertex.point[0] < xMax and yMin < event.vertex.point[1] < yMax and zMin < event.vertex.point[2] < zMax):
                if( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                    trackSide=0
                    numClusters+=1
                    if (cluster.detector in alpidesLeft): #remember we are defining a normal vector for the cylinder. here we get the detector and have a radius
                        #trackSide=1

                        normalVector=[cluster.globalPos[0],cluster.globalPos[1],0 ] #cluster coords perpendicular to z
                        normalVector/=np.linalg.norm(normalVector)
                        trackVector=event.tracks[trackSide].vector/np.linalg.norm(event.tracks[trackSide].vector) #np.linalg.norm should return the magnitude
                        pathLength=0.025/np.dot(normalVector,trackVector)*1000 #get hypotenus (track length through det)convert mm to micron
                        #pathLenSum+=np.abs(pathLength) 
                        histograms[hist].Fill(cluster.size, np.abs(pathLength),1/volume) #,cluster.size, 1/tVolume)
    

                    
#Draw histogram

canvas.Divide(3)
for index,hist in enumerate(histograms):
    canvas.cd(index+1)
    hist.Draw("COLZ")
    hist.SetXTitle("Cluster Size / [pixels]")
    hist.SetYTitle("Track Length through ALPIDE / Micron")
canvas.Draw()



In [ ]:
# Create profile
hProfile1 = hCluster1.ProfileX()
canvas = ROOT.TCanvas()

hProfile1.Draw()
canvas.Draw()

## Cluster Size Fit Mean vs. Energy

In [ ]:
%jsroot on
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]
colors = [2,3,4,6,7,8,9]
canvas = ROOT.TCanvas("","",600,600)
legend = ROOT.TLegend()

# Calculate histogram

hCluster1 = ROOT.TH2F("Cluster_Size", "Cluster Size Fit Mean vs. Energy", 300, 70, 210, 21, -0.5, 20.5)
hCluster2 = ROOT.TH2F("Cluster_Size", "", 300, 70, 210, 21, -0.5, 20.5)
hCluster3 = ROOT.TH2F("Cluster_Size", "", 300, 70, 210.5, 21, -0.5, 20.5)

graph1 = ROOT.TGraphErrors()
graph2 = ROOT.TGraphErrors()
graph3 = ROOT.TGraphErrors()

histograms = [hCluster1, hCluster2, hCluster3] #three beam energies
graphs=[graph1, graph2, graph3]

energies=[200, 120, 80]

sigmas1=[1.8,2.1,2.1]
sigmas2=[1.7,1.7,1.8]
sigmas3=[0,1.5,0]
sigmas= [sigmas1,sigmas2,sigmas3]

fitMeanClusterSize100e=[9.9, 13, 15]
fitMeanClusterSize200e=[7.9, 10, 12]
fitMeanClusterSize300e=[-1, 8.2, -1]
fitMeanClusterSizes = [fitMeanClusterSize100e, fitMeanClusterSize200e, fitMeanClusterSize300e]

for thresh, meanClustSize in enumerate(fitMeanClusterSizes):
    for energy, meanSize in enumerate(meanClustSize):
        #histograms[thresh].Fill(energies[energy], meanSize)
        graphs[thresh].SetPoint(energy, energies[energy], meanSize)
        graphs[thresh].SetPointError(energy, 0, sigmas[thresh][energy])
    
                    
#Draw histogram
labels=["100e", "200e","300e"]
det=0


for index,hist in enumerate(histograms):
    #hist[det].Scale(1/hist[det].Integral())
    graphs[index].SetMinimum(5)
    
    hist.SetMarkerColor(colors[index])
    hist.SetMarkerStyle(20)
    graphs[index].SetMarkerColor(colors[index])
    graphs[index].SetMarkerStyle(20)
    hist.Draw("same P L")
    graphs[index].Draw("same P")
    ROOT.gPad.SetGrid(2)
    hist.SetXTitle("Beam Energy / MeV")
    hist.SetYTitle("Mean Cluster Size / pixels")
    legend.AddEntry(graphs[index], labels[index])
    
    
    
legend.Draw("COL") 
canvas.Draw("COLZ")